<a href="https://colab.research.google.com/github/Onkar-stac/Next-Word-Predictor/blob/main/nextwordpredictor_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import library
import re
import requests
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [ ]:
# data to extract
url = "https://www.gutenberg.org/cache/epub/5200/pg5200.txt"

In [ ]:
#function to extract data from the url
def get_book(url):
  raw = requests.get(url).text
  #discarding the beginning of the data
  start = re.search(r"\*\*\* START OF THIS PROJECT GUTENBERG EBOOK .* \*\*\*",raw).end()
  #discarding the end of the data
  stop=re.search(r"II", raw).start()
  text=raw[start:stop] #relevant data
  return text

In [ ]:
#processing
def preprocess(sentence):
  return re.sub('[^A-Za-z0-9.]+',' ',sentence).lower()

In [ ]:
#calling the functions
book = get_book(url)
processed_book=preprocess(book)
print(processed_book)

 copyright c 2002 david wyllie. metamorphosis franz kafka translated by david wyllie i one morning when gregor samsa woke from troubled dreams he found himself transformed in his bed into a horrible vermin. he lay on his armour like back and if he lifted his head a little he could see his brown belly slightly domed and divided by arches into stiff sections. the bedding was hardly able to cover it and seemed ready to slide off any moment. his many legs pitifully thin compared with the size of the rest of him waved about helplessly as he looked. what s happened to me he thought. it wasn t a dream. his room a proper human room although a little too small lay peacefully between its four familiar walls. a collection of textile samples lay spread out on the table samsa was a travelling salesman and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice gilded frame. it showed a lady fitted out with a fur hat and fur boa who sat upright rais

In [ ]:
len(processed_book)

37409

In [ ]:
# EDA
len(re.findall(r'the',processed_book))
processed_book = re.sub(r'\si\s', " I ",processed_book)
processed_book = re.sub(r'[^\w\s]'," ",processed_book)
print(processed_book)

 copyright c 2002 david wyllie  metamorphosis franz kafka translated by david wyllie I one morning when gregor samsa woke from troubled dreams he found himself transformed in his bed into a horrible vermin  he lay on his armour like back and if he lifted his head a little he could see his brown belly slightly domed and divided by arches into stiff sections  the bedding was hardly able to cover it and seemed ready to slide off any moment  his many legs pitifully thin compared with the size of the rest of him waved about helplessly as he looked  what s happened to me he thought  it wasn t a dream  his room a proper human room although a little too small lay peacefully between its four familiar walls  a collection of textile samples lay spread out on the table samsa was a travelling salesman and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice gilded frame  it showed a lady fitted out with a fur hat and fur boa who sat upright rais

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([processed_book])
#saving the tokenizer
pickle.dump(tokenizer,open('token.pkl','wb'))
sequence_data = tokenizer.texts_to_sequences([processed_book])[0] #vectorize the text corpus
sequence_data[:15]

[642, 643, 644, 421, 422, 645, 646, 647, 648, 75, 421, 422, 15, 57, 163]

In [ ]:
len(sequence_data)

7275

In [ ]:
vocab_size = len(tokenizer.word_index)+1 #number of unique words in the text corpus
print(vocab_size)

1355


In [ ]:
sequences = []
for i in range(3,len(sequence_data)):
  words = sequence_data[i-3:i+1]
  sequences.append(words)
print("The length of sequences are: ",len(sequences))
sequences = np.array(sequences)
sequences[:10]

The length of sequences are:  7272


array([[642, 643, 644, 421],
       [643, 644, 421, 422],
       [644, 421, 422, 645],
       [421, 422, 645, 646],
       [422, 645, 646, 647],
       [645, 646, 647, 648],
       [646, 647, 648,  75],
       [647, 648,  75, 421],
       [648,  75, 421, 422],
       [ 75, 421, 422,  15]])

In [ ]:
X=[]
Y=[]
for i in sequences:
  X.append(i[0:3])
  Y.append(i[3])
X=np.array(X)
Y=np.array(Y)


In [ ]:
Y=to_categorical(Y,num_classes=vocab_size)
Y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size,10,input_length=3))
model.add(LSTM(1000))
model.add(Dropout(0.2))
model.add(Dense(vocab_size,activation="softmax"))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             13550     
                                                                 
 lstm (LSTM)                 (None, 1000)              4044000   
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 1355)              1356355   
                                                                 
Total params: 5,413,905
Trainable params: 5,413,905
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
file_name_path="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(file_name_path, monitor='loss', 
verbose=1, save_best_only=True, mode='min')
model.compile(loss="categorical_crossentropy",optimizer=Adam(learning_rate=0.001))
callbacks = [checkpoint]

In [ ]:
model.fit(X, Y, epochs=50, batch_size=64, callbacks=callbacks) 

Epoch 1/50
114/114 [==============================] - ETA: 0s - loss: 6.2920
Epoch 00001: loss improved from inf to 6.29201, saving model to weights-improvement-01-6.2920.hdf5
114/114 [==============================] - 7s 15ms/step - loss: 6.2920
Epoch 2/50
112/114 [============================>.] - ETA: 0s - loss: 5.8976
Epoch 00002: loss improved from 6.29201 to 5.89983, saving model to weights-improvement-02-5.8998.hdf5
114/114 [==============================] - 2s 14ms/step - loss: 5.8998
Epoch 3/50
112/114 [============================>.] - ETA: 0s - loss: 5.8514
Epoch 00003: loss improved from 5.89983 to 5.85193, saving model to weights-improvement-03-5.8519.hdf5
114/114 [==============================] - 2s 15ms/step - loss: 5.8519
Epoch 4/50
114/114 [==============================] - ETA: 0s - loss: 5.7851
Epoch 00004: loss improved from 5.85193 to 5.78508, saving model to weights-improvement-04-5.7851.hdf5
114/114 [==============================] - 2s 14ms/step - loss: 5.7851


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle
model = load_model('weights-improvement-50-0.6095.hdf5')
tokenizer = pickle.load(open('token.pkl','rb'))

def Predict_Next_Words(model,tokenizer,text):
  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""

  for key,value in tokenizer.word_index.items():
    if value==preds:
      predicted_word=key
      break
  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter: ")
  if text=="0":
    print("Execution completed")
    break
  else:
    text=text.split(" ")
    text=text[-3:]
    print(text)
    Predict_Next_Words(model,tokenizer,text)

Enter: A collection of textile
['collection', 'of', 'textile']
samples
Enter: Samsa was a travelling
['was', 'a', 'travelling']
and
Enter: he had recently cut out of an
['out', 'of', 'an']
illustrated
Enter: 0
Execution completed
